In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [15]:
num_epochs=4
batch_size=4
lr=0.001

In [20]:
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [22]:
train_loader=torch.utils.data.DataLoader(train_data,batch_size=100,shuffle=True, num_workers=1)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=100,shuffle=True,num_workers=1)


In [45]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

In [46]:
cnn = CNN()
print(cnn)


CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [47]:
loss_func=nn.CrossEntropyLoss()
loss_func

CrossEntropyLoss()

In [48]:
from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)   
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [53]:
from torch.utils.data import DataLoader
loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x2be67343c70>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x2be67341f90>}

In [54]:
from torch.autograd import Variable
num_epochs = 10
def train(num_epochs, cnn, loaders):
    
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
             
train(num_epochs, cnn, loaders)

Epoch [1/10], Step [100/600], Loss: 0.2639
Epoch [1/10], Step [200/600], Loss: 0.1087
Epoch [1/10], Step [300/600], Loss: 0.0988
Epoch [1/10], Step [400/600], Loss: 0.0195
Epoch [1/10], Step [500/600], Loss: 0.0405
Epoch [1/10], Step [600/600], Loss: 0.1518
Epoch [2/10], Step [100/600], Loss: 0.0064
Epoch [2/10], Step [200/600], Loss: 0.0842
Epoch [2/10], Step [300/600], Loss: 0.1168
Epoch [2/10], Step [400/600], Loss: 0.1128
Epoch [2/10], Step [500/600], Loss: 0.0711
Epoch [2/10], Step [600/600], Loss: 0.0246
Epoch [3/10], Step [100/600], Loss: 0.1228
Epoch [3/10], Step [200/600], Loss: 0.0359
Epoch [3/10], Step [300/600], Loss: 0.0224
Epoch [3/10], Step [400/600], Loss: 0.0299
Epoch [3/10], Step [500/600], Loss: 0.0164
Epoch [3/10], Step [600/600], Loss: 0.0432
Epoch [4/10], Step [100/600], Loss: 0.0297
Epoch [4/10], Step [200/600], Loss: 0.0413
Epoch [4/10], Step [300/600], Loss: 0.0617
Epoch [4/10], Step [400/600], Loss: 0.0964
Epoch [4/10], Step [500/600], Loss: 0.0657
Epoch [4/10